In [1]:
import geopandas as gpd
import pandas as pd
from zipfile import ZipFile

import os
os.chdir("../")

In [14]:
year = 2021
election_round = "general"

In [3]:
df = pd.read_csv(ZipFile(f"data/Germany/{year}.zip").open("btw21_wbz_ergebnisse.csv"), delimiter=";")
df = df[~df["Land"].isna()]

df["polling_id"] = df["Wahlkreis"].astype(int).astype(str) + df["Land"].astype(int).astype(str) \
    + df["Gemeinde"].astype(int).astype(str) + df["Wahlbezirk"].astype(str) \
+ df["Bezirksart"].astype(int).astype(str)

df.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_19540\3449718750.py:1: DtypeWarning: Columns (6,7,9,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ZipFile(f"data/Germany/{year}.zip").open("btw21_wbz_ergebnisse.csv"), delimiter=";")


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Urnenwahlbezirke nach § 68 BWO,Kennziffer Briefwahlzugehörigkeit,Gemeinde Name,Wahlbezirk,...,LIEBE.1,LKR.1,PdF,LfK,SSW.1,Team Todenhöfer.1,UNABHÄNGIGE.1,Volt.1,Ungekürzte Wahlbezirksbezeichnung,polling_id
1,1.0,1.0,0.0,1.0,0.0,0.0,0000,0.0,"Flensburg, Stadt",01,...,0,0,0,0,118,5,0,1,01,110010
2,1.0,1.0,0.0,1.0,0.0,0.0,0000,0.0,"Flensburg, Stadt",02,...,0,0,0,0,130,13,0,3,02,110020
3,1.0,1.0,0.0,1.0,0.0,0.0,0000,0.0,"Flensburg, Stadt",03,...,0,0,0,0,106,4,0,0,03,110030
4,1.0,1.0,0.0,1.0,0.0,0.0,0000,0.0,"Flensburg, Stadt",04,...,0,0,0,0,84,6,0,0,04,110040
5,1.0,1.0,0.0,1.0,0.0,0.0,0000,0.0,"Flensburg, Stadt",05,...,0,0,0,0,50,3,0,1,05,110050


In [4]:
df["lau_id"] = df["Land"].astype(int).astype(str).str.zfill(2) \
    + df["Regierungsbezirk"].astype(int).astype(str).str.zfill(1) \
    + df["Kreis"].astype(int).astype(str).str.zfill(2) \
    + df["Gemeinde"].astype(int).astype(str).str.zfill(3)

In [5]:
party = ["CDU", "SPD", "AfD", "FDP", "DIE LINKE", "GRÜNE", "CSU", "FREIE WÄHLER", "Die PARTEI", "Tierschutzpartei", "NPD", "PIRATEN", "ÖDP", "V-Partei³", "DiB", "BP", "Tierschutzallianz", "MLPD", "Gesundheitsforschung", "MENSCHLICHE WELT", "DKP", "Die Grauen", "BüSo", "Die Humanisten", "Gartenpartei", "du.", "dieBasis", "Bündnis C", "BÜRGERBEWEGUNG", "III. Weg", "Bündnis21", "LIEBE", "LKR", "SSW", "Team Todenhöfer", "UNABHÄNGIGE", "Volt", "Volksabstimmung", "B*", "sonstige", "FAMILIE", "Graue Panther", "KlimalisteBW", "THP"]

In [6]:
df_filtered = df[["polling_id"] + party].melt(id_vars=["polling_id"], var_name="candidate")
df_filtered["value"] = df_filtered["value"].fillna(0).astype(int)
df_filtered.head()

,polling_id,candidate,value
0,110010,CDU,129
1,110020,CDU,114
2,110030,CDU,76
3,110040,CDU,54
4,110050,CDU,27


In [7]:
tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

In [8]:
df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "candidate"])

In [15]:
df_filtered.to_csv(f"data_output/Germany/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [16]:
df = df.rename(columns={
    "Wahlkreis": "constituency",
    "Gemeinde Name": "municipality",
    "Gemeinde": "municipality_id"
})

df_lau = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/501971/EU-27-LAU-2022-NUTS-2021.xlsx",
    sheet_name="DE"
)
df_lau = df_lau[["NUTS 3 CODE", "LAU CODE"]].rename(columns={"NUTS 3 CODE": "nuts_3", "LAU CODE": "lau_id"})
df_lau["nuts_2"] = df_lau["nuts_3"].str[0:4]
df_lau["nuts_1"] = df_lau["nuts_3"].str[0:3]
df_lau["lau_id"] = df_lau["lau_id"].astype(str).str.zfill(8)

df_location = df[["polling_id", "constituency", "lau_id", "municipality", "municipality_id"]].drop_duplicates()
df_location = pd.merge(df_location, df_lau, on="lau_id", how="left")

df_location.to_csv(f"data_output/Germany/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)